In [ ]:
from wikibaseintegrator.models import Qualifiers, References, Reference
from wikibaseintegrator import WikibaseIntegrator
from wikibaseintegrator import wbi_login
from wikibaseintegrator import datatypes
from wikibaseintegrator.wbi_config import config as wbi_config
from os.path import join, isfile, exists, isdir, abspath, pardir
from os import listdir, getcwd
import pandas as pd
from wikibaseintegrator.datatypes import (URL, CommonsMedia, ExternalID, Form, GeoShape, GlobeCoordinate, Item, Lexeme, Math, MonolingualText, MusicalNotation, Property, Quantity, Sense, String, TabularData, Time)
from wikibaseintegrator.wbi_enums import ActionIfExists, WikibaseDatePrecision, WikibaseRank, WikibaseSnakType
from wikibaseintegrator.wbi_exceptions import MissingEntityException, ModificationFailed, MWApiError

In [ ]:
# Default variables
WDUSER, WDPASS = "Admin", "90J8XXXaO4Sr9^^Z"
# WDUSER, WDPASS = "Mohammadzainabbas", "fHh!%shFa6^h"

In [ ]:
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, "data")
team_data = join(data_dir, "BDMA_Team_v1.csv")
project_data = join(data_dir, "BDMA_Projects_v1.csv")

In [ ]:
def setup_config():
    """
    Set up WBI config to use local docker installation
    """
    wbi_config['MEDIAWIKI_API_URL'] = 'http://139.144.66.193:8181/api.php'
    wbi_config['SPARQL_ENDPOINT_URL'] = '"http://139.144.66.193:8282/proxy/wdqs/bigdata/namespace/wdq/sparql"'
    wbi_config['WIKIBASE_URL'] = 'http://wikibase.svc'

In [ ]:
setup_config()

In [ ]:
login = wbi_login.Login(user=WDUSER, password=WDPASS)

In [ ]:
wbi = WikibaseIntegrator(login=login)

In [ ]:
df = pd.read_csv(team_data)
df.head()

In [ ]:
column_names = df.columns.to_list()

In [ ]:
def create_mapping(file_name):
    # Read .csv file to create mappings
    __df = pd.read_csv(join(file_name))
    
    if __df.empty: return

    columns_with_mappings = list(__df.T.to_dict().values())

    for item in columns_with_mappings:
        __col_name = item['column']
        __data_type = item['data_type']
        __type = item['type']
        __mapping_code = item['mapping_code']
        __description = item['description']
        __alias = item['alias']

        try:

            p = wbi.property.new(datatype=__data_type)
            p.labels.set('en', __col_name)
            p.descriptions.set('en', __description)
            p.aliases.set('en', __alias)

            res = p.write()

            ident = [x for x in str(res).split('\n') if "_id='P" in x]
            if len(ident) == 1:
                prop_code = ident[0].split("'")[1]
                print("Column '{}' is mapped to '{}'".format(__col_name, prop_code))
            else:
                raise Exception("Surprise, this method didn't work.")
        
        except ModificationFailed as e:
            print("Property '{}' already exists".format(__col_name))
            continue

In [ ]:
team_mapping = join(data_dir, "team_mapping.csv")
create_mapping(team_mapping)

In [ ]:
project_mapping = join(data_dir, "project_mapping.csv")
create_mapping(project_mapping)